In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import datetime
import re

In [2]:
def get_next_page(driver):
    try:
        driver.find_element_by_class_name("ui_button.nav.next.primary").send_keys(Keys.ENTER)
        return True
    except:
        print('마지막 페이지입니다')
        return False

In [3]:
def convert_date(date):
    stay_date = date.split('숙박 날짜: ')
    stay_date = stay_date[1]

    # 20XX년 X월 형태의 str
    stay_date = stay_date.replace('년', '').replace(' ', '').replace('월', '')
    dt = datetime.datetime.strptime(stay_date, '%Y%m')
    dt = dt.date() #20XX-XX-01 형태의 date 객체
    return dt

In [23]:
# convert_date 실행 예시
convert_date('숙박 날짜: 2020년 11월')

datetime.date(2020, 11, 1)

In [4]:
def get_data(driver):
    iteration = True
    df = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])
    try:
        time.sleep(20)
        #WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        #                                                                    "div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu")))
        # 더보기 클릭
        driver.find_element_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu").click()

        time.sleep(5)
        # 숙박 날짜 가져오기
        dates = driver.find_elements_by_class_name("_34Xs-BQm")
        stay_dates = []
        for d in dates:
            stay_date = convert_date(d.text)
            stay_dates.append(stay_date)
            # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
            if stay_date < datetime.date(2018,6,1): 
                iteration = False

        # 닉네임 가져오기
        nicknames = driver.find_elements_by_css_selector("div._310S4sqz > div > div._2fxQ4TOx > span > a")
        # 별점 가져오기
        rating_codes = driver.find_elements_by_css_selector("div.oETBfkHU > div._2UEC-y30 > div > span")
        scores = []
        for rc in rating_codes:
            cls_attr = str(rc.get_attribute("class")).split("ui_bubble_rating bubble_")
            score = int(cls_attr[1])
            scores.append(score)

        # 리뷰 가져오기
        reviews = driver.find_elements_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu > div.cPQsENeY > q > span")
        
        for i in range(len(nicknames)):
            df.loc[i] = [nicknames[i].text, scores[i], reviews[i].text, stay_dates[i]]
            #print(df.loc[i])

    except ElementClickInterceptedException as e:
        print(e)
    except StaleElementReferenceException as e:
        print(e)

    finally:
        return df, iteration

In [5]:
def hotel_review_crawling(url):
    # 데이터 수집
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=options)

    data = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])

    #첫페이지 시작
    #데이터 크롤링, df에 추가하기
    #다음페이지 클릭 -> disbaled 나오기 전까지 반복

    driver.get(url)
    driver.implicitly_wait(10)
    iteration = True
    page = 1
    while iteration:
        print(page, '번째 페이지')
        tmp_df, iteration = get_data(driver)
        data = pd.concat([data, tmp_df], axis=0, ignore_index=True)
        if iteration == False:
            break
        iteration = get_next_page(driver) 
        page += 1

    #data.to_csv('./hotel_review_test_sillastay_mapo.csv', encoding="utf-8-sig")
    
    return data

In [6]:
def preprocessing_data(data):
    # 개행문자 제거
    for idx, review in enumerate(data['review']):
        data['review'][idx] = review.replace('\n', ' ')
    # 날짜순으로 내림차순 정렬
    data.sort_values(by='stay_date', ascending=False, 
                     ignore_index=True, inplace=True)
    # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
    data = data[data['stay_date'] > datetime.date(2018,5,1)]
    # 특수문자, 이모지 제거
    ###
    return data

In [7]:
# 실제로 실행하는 코드
url = "https://www.tripadvisor.co.kr/Hotel_Review-g297884-d12242451-Reviews-Shilla_Stay_Haeundae-Busan.html"

data_sample = hotel_review_crawling(url)
data_sample

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지
7 번째 페이지
8 번째 페이지
9 번째 페이지
10 번째 페이지
11 번째 페이지
12 번째 페이지
13 번째 페이지
14 번째 페이지
15 번째 페이지
16 번째 페이지
17 번째 페이지
18 번째 페이지
19 번째 페이지
20 번째 페이지
21 번째 페이지
22 번째 페이지
23 번째 페이지
24 번째 페이지
25 번째 페이지
26 번째 페이지
27 번째 페이지
28 번째 페이지
29 번째 페이지
30 번째 페이지
31 번째 페이지
32 번째 페이지
33 번째 페이지
34 번째 페이지
35 번째 페이지
36 번째 페이지
37 번째 페이지
38 번째 페이지
39 번째 페이지
40 번째 페이지
41 번째 페이지
42 번째 페이지
43 번째 페이지
44 번째 페이지
45 번째 페이지
46 번째 페이지
47 번째 페이지
48 번째 페이지
49 번째 페이지
50 번째 페이지
51 번째 페이지
52 번째 페이지
53 번째 페이지
54 번째 페이지
55 번째 페이지
56 번째 페이지
57 번째 페이지
58 번째 페이지
59 번째 페이지
60 번째 페이지
61 번째 페이지
62 번째 페이지
63 번째 페이지
64 번째 페이지
65 번째 페이지
66 번째 페이지
67 번째 페이지
68 번째 페이지
69 번째 페이지
70 번째 페이지
71 번째 페이지
72 번째 페이지
73 번째 페이지
74 번째 페이지
75 번째 페이지
76 번째 페이지
77 번째 페이지
78 번째 페이지
79 번째 페이지
80 번째 페이지
81 번째 페이지
82 번째 페이지
83 번째 페이지
84 번째 페이지
85 번째 페이지
86 번째 페이지
87 번째 페이지
88 번째 페이지
89 번째 페이지
90 번째 페이지
91 번째 페이지
92 번째 페이지
93 번째 페이지
94 번째 페이지
95 번째 페이지
96 번째 페이지
97 번째 페이지
98 번째 페이지
99 번째 페이지
100 번째 페이지
101 번째 페

,nickname,score,review,stay_date
0,금태현,50,호텔 직원분들 너무 친절하시고 호텔 내부도 너무 깔끔합니다. 기회가되면 다시 방문하...,2021-06-01
1,써니,50,해운대바로앞이라 위치 최상입니다. 직원분들 너무 친절하시고 룸컨디션 깨끗하고 좋아요...,2021-06-01
2,승연 김,50,가격도 좋고 뷰도 좋고 친구들이랑 오랜만에 놀려구 예약했어요~ 주변에 맛집도 많아서...,2021-06-01
3,나효수,50,룸컨디션 좋고 잠도 편안히 잘수 있어서 좋앗습니다 호텔 가성비로는 해운대앞애서 는 ...,2021-05-01
4,JUNGHYUN L,50,깔끔하고 최신 시설\n인테리어도 깔끔하고 모던한 디자인\n신라호텔만의 특급서비스\n...,2021-06-01
...,...,...,...,...
1270,skyGyu,50,여자친구와 방문한 신라스테이~\n호텔에서 바다를 볼수있어서 너무 좋았고 무엇보다 루...,2018-08-01
1271,jhjung26,40,해운대에 위치하고 있는 호텔 중 가성비가 가장 좋지 않을까 생각됨. 부산모터쇼 기간...,2018-05-01
1272,유빈 김,40,"가족과 잊을 수 없는 추억 만들고 갑니다.\n직원들도 친절하고 호텔 시설로, 깔끔한...",2018-07-01
1273,유림 진,40,건물자체가 깔끔하고 방도 아늑하고 뷰도 너무너무 맘에 들었다ㅎ\n한가지 아쉬운점은 ...,2018-07-01


In [10]:
# 중간에 크롤링이 끊겼을 경우
url_next = "https://www.tripadvisor.co.kr/Hotel_Review-g297884-d12242451-Reviews-or1305-Shilla_Stay_Haeundae-Busan.html"
data_sample2 = hotel_review_crawling(url_next)

data_sample_concat = pd.concat([data_sample_concat, data_sample2], axis=0, ignore_index=True)
data_sample_concat

1 번째 페이지
2 번째 페이지


,nickname,score,review,stay_date
0,금태현,50,호텔 직원분들 너무 친절하시고 호텔 내부도 너무 깔끔합니다. 기회가되면 다시 방문하...,2021-06-01
1,써니,50,해운대바로앞이라 위치 최상입니다. 직원분들 너무 친절하시고 룸컨디션 깨끗하고 좋아요...,2021-06-01
2,승연 김,50,가격도 좋고 뷰도 좋고 친구들이랑 오랜만에 놀려구 예약했어요~ 주변에 맛집도 많아서...,2021-06-01
3,나효수,50,룸컨디션 좋고 잠도 편안히 잘수 있어서 좋앗습니다 호텔 가성비로는 해운대앞애서 는 ...,2021-05-01
4,JUNGHYUN L,50,깔끔하고 최신 시설\n인테리어도 깔끔하고 모던한 디자인\n신라호텔만의 특급서비스\n...,2021-06-01
...,...,...,...,...
1310,Seunghyun J,50,돈을 조금더 추가해서 씨뷰로 했는데 너무좋아요~\n어메니티 아베다고 침구 깔끔 푹신...,2018-06-01
1311,soram83,50,생긴지 오래되지않아 시설이 깨끗하고 좋아요! 가성비 최고입니다. 해운대 해변도 가깝...,2018-06-01
1312,진희 한,50,너무 깔끔하고 깨끗합니다. 생일날 왔는데 최고로 좋습니다 다음에도 꼬 오고싶은 호텔...,2018-06-01
1313,조무현 J,50,예전에 와 봤던 해운대~아들한테 바다보여주고 싶어서 다시 찾았네요ㅎㅎ숙소를 찾던중 ...,2018-06-01


In [11]:
data_sample_concat

,nickname,score,review,stay_date
0,금태현,50,호텔 직원분들 너무 친절하시고 호텔 내부도 너무 깔끔합니다. 기회가되면 다시 방문하...,2021-06-01
1,써니,50,해운대바로앞이라 위치 최상입니다. 직원분들 너무 친절하시고 룸컨디션 깨끗하고 좋아요...,2021-06-01
2,승연 김,50,가격도 좋고 뷰도 좋고 친구들이랑 오랜만에 놀려구 예약했어요~ 주변에 맛집도 많아서...,2021-06-01
3,나효수,50,룸컨디션 좋고 잠도 편안히 잘수 있어서 좋앗습니다 호텔 가성비로는 해운대앞애서 는 ...,2021-05-01
4,JUNGHYUN L,50,깔끔하고 최신 시설\n인테리어도 깔끔하고 모던한 디자인\n신라호텔만의 특급서비스\n...,2021-06-01
...,...,...,...,...
1310,Seunghyun J,50,돈을 조금더 추가해서 씨뷰로 했는데 너무좋아요~\n어메니티 아베다고 침구 깔끔 푹신...,2018-06-01
1311,soram83,50,생긴지 오래되지않아 시설이 깨끗하고 좋아요! 가성비 최고입니다. 해운대 해변도 가깝...,2018-06-01
1312,진희 한,50,너무 깔끔하고 깨끗합니다. 생일날 왔는데 최고로 좋습니다 다음에도 꼬 오고싶은 호텔...,2018-06-01
1313,조무현 J,50,예전에 와 봤던 해운대~아들한테 바다보여주고 싶어서 다시 찾았네요ㅎㅎ숙소를 찾던중 ...,2018-06-01


In [12]:
# 기본 전처리 전 원본 데이터 저장(유실방지)
data_sample_concat.to_csv('./data/hotel_review_shillastay_haeundae_origin.csv', encoding="utf-8-sig")

In [13]:
data = preprocessing_data(data_sample_concat)

In [14]:
data

,nickname,score,review,stay_date
0,금태현,50,호텔 직원분들 너무 친절하시고 호텔 내부도 너무 깔끔합니다. 기회가되면 다시 방문하...,2021-06-01
1,Aram,50,직원분들도 친절하시도 깔끔하니 너무 좋습니다 시원하고 향기도 좋고 역시 신라스테이 ...,2021-06-01
2,kim si eun,50,해운대 바다 바로 앞이고 지하철역이랑 가까워서 편합니다!! 깔끔하고 쾌적한호텔을 찾...,2021-06-01
3,WONNY H,50,친구들이랑 같이 왔는데 깨끗하고 쾌적해요 ! 합리적인 가격에 묵을 수 있을 수 있어...,2021-06-01
4,써니,50,해운대바로앞이라 위치 최상입니다. 직원분들 너무 친절하시고 룸컨디션 깨끗하고 좋아요...,2021-06-01
...,...,...,...,...
1306,jihwan1962,50,하루 잘 머물다가 갑니다. 가족끼리 잘 지내다 가네요 꼭 추천드리고 해운대에 오실때...,2018-06-01
1307,SOYEONG S,50,바다보면서 힐링하러 온건데 만족스럽습니다. 스텝분들도 친절하셔서 좋은 기분으로 안내...,2018-06-01
1308,한숙 최,50,해운대 오션뷰 너무 좋아요!! 오늘 날씨가 흐려서 아쉽지만 그래도 확 트인 창이 너...,2018-06-01
1309,가늬,50,여름휴가가없어 미리 해운대로 놀러와서 신라스테이 숙박을하였는데 위치 분위기 직원친절...,2018-06-01


In [15]:
# csv 저장을 위한 date to str convert
data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))
# 기본 전처리후 데이터 csv 파일로 저장 
#data.to_csv('./data/hotel_review_shillastay_yeoksam.csv', encoding="utf-8-sig")

<ipython-input-15-7183346d9643>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))


In [16]:
# 리뷰 내 이모지 제거
data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))
#data.to_csv('./data/hotel_review_shillastay_yeoksam_del_emoji.csv', encoding="utf-8-sig")

<ipython-input-16-b71e843350f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))


In [17]:
# 리뷰 내 특수문자 제거
test_data = data
special = re.compile(r'[^ A-Za-z0-9가-힣+]')
test_data['review'] = [special.sub('', s) for s in test_data['review']]

# 특수문자 제거 후 파일 저장
test_data.to_csv('./data/hotel_review_shillastay_haeundae_final.csv', encoding="utf-8-sig")

<ipython-input-17-c5250b68e6f1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['review'] = [special.sub('', s) for s in test_data['review']]


In [18]:
test_data

,nickname,score,review,stay_date
0,금태현,50,호텔 직원분들 너무 친절하시고 호텔 내부도 너무 깔끔합니다 기회가되면 다시 방문하고...,2021-06-01
1,Aram,50,직원분들도 친절하시도 깔끔하니 너무 좋습니다 시원하고 향기도 좋고 역시 신라스테이 ...,2021-06-01
2,kim si eun,50,해운대 바다 바로 앞이고 지하철역이랑 가까워서 편합니다 깔끔하고 쾌적한호텔을 찾으신...,2021-06-01
3,WONNY H,50,친구들이랑 같이 왔는데 깨끗하고 쾌적해요 합리적인 가격에 묵을 수 있을 수 있어서...,2021-06-01
4,써니,50,해운대바로앞이라 위치 최상입니다 직원분들 너무 친절하시고 룸컨디션 깨끗하고 좋아요 ...,2021-06-01
...,...,...,...,...
1306,jihwan1962,50,하루 잘 머물다가 갑니다 가족끼리 잘 지내다 가네요 꼭 추천드리고 해운대에 오실때는...,2018-06-01
1307,SOYEONG S,50,바다보면서 힐링하러 온건데 만족스럽습니다 스텝분들도 친절하셔서 좋은 기분으로 안내받...,2018-06-01
1308,한숙 최,50,해운대 오션뷰 너무 좋아요 오늘 날씨가 흐려서 아쉽지만 그래도 확 트인 창이 너무 ...,2018-06-01
1309,가늬,50,여름휴가가없어 미리 해운대로 놀러와서 신라스테이 숙박을하였는데 위치 분위기 직원친절...,2018-06-01
